In [ ]:
from getpass import getpass
import warnings

from pandas.io.pytables import format_doc
from prompt_toolkit.utils import to_str

warnings.filterwarnings('ignore')
from utils import ChatOpenAI

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [24]:
import pandas as pd
from langchain.document_loaders import CSVLoader, DataFrameLoader, PyPDFLoader

doc1 = pd.read_csv("https://stepik.org/media/attachments/lesson/1084288/pushkin_questions.csv")
# loader_qu = DataFrameLoader(doc1, page_content_column="question")
# doc_qu = loader_qu.load()
# doc_qu
doc1

,question
0,Кого убили раздетой на крыльце?
1,Где Маша остановилась в Петербурге?
2,Почему Гринев и Савельич сбились с дороги?
3,Кто издевался над Машей и морил ее голодом?
4,Куда приговорили отправить Гринева в ссылку?
5,Сколько детей было в семье Петра Гринева?
6,Кто воспитывал Петра Гринева?
7,"Сколько казаков попросил Гринев, чтобы очистит..."
8,Как далеко находилась от города Белогорская кр...
9,Как звали дочку коменданта?


In [ ]:
# Определим лоадер pdf
loader = PyPDFLoader(file_path="R:\AProgr\AI\AI_проекты\AI_project_1701\data\The_Daughter_of_The_Commandant.pdf")
documents_pdf = loader.load()

# Посмотрим как выглядит документ
# documents_pdf = documents_pdf[0].page_content
doc_pdf = []
for i in documents_pdf:
    doc_pdf.append(str(i)) # перевожу все элементы в str т.к была ошибка(ожидание строки или битов)
 
ggg = ' ' 
for i in doc_pdf:
    ggg =ggg+i    
    
    
print(ggg)
type(ggg)

In [ ]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)


split_documents = splitter.create_documents([ggg])
len(split_documents)

In [ ]:
# # Если используете ключ курса, запустите эту ячейку
# from utils import OpenAIEmbeddings
# 
# embeddings_api_model = OpenAIEmbeddings(course_api_key=course_api_key)

In [ ]:


# embeddings = embeddings_api_model.embed_documents(
#     [
#        str_1
#     ]
# )
# # Посмотрим размерность эмбеддингов
# print(len(embeddings), len(embeddings[0]), type(embeddings), type(embeddings[0]))
# embeddings[0][:5]

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'                                          или cuda - если через видеокарту
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)


# print(len(embeddings), len(embeddings[0]))

In [ ]:
from langchain.vectorstores import FAISS
from utils import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(course_api_key=course_api_key)
db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)  

db.save_local("faiss_db") # можно сохранить базу локально, указав путь


In [ ]:
# Также можно вывести скор каждого документа
db.similarity_search_with_score(
    "Кого убили раздетой на крыльце?"
)

In [ ]:
                                                  #  1 вариант retriever(извлечение)
#  Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
# retriever.get_relevant_documents(
#     "Кого убили раздетой на крыльце?"
# )


In [ ]:
                                            #  2 вариант retriever(извлечение)

from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Попробуем BM25 - алгоритм поиска по ключевым словам. Может искать лучше, но не поймёт запрос на другом языке.
# Также не сможет искать, если запрос на обывательском языке, а документы на профессиональном
bm25 = BM25Retriever.from_documents(split_documents[:70])  # Эмбеддинги ему не нужны
bm25.k = 5  # Так можно задать количество возвращаемых документов

bm25.get_relevant_documents(
    "Кого убили раздетой на крыльце?"
)[:1]

In [ ]:
                                    #  3 вариант retriever(извлечение) Объединяем bm25 и retriever (первый и второй вариант)
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# Если один ретривер плохо справляется, то можно использовать целый ансамбль и даже каждому вес назначить
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],  # список ретриверов
    weights=[
        0.4,
        0.6,
    ],  # веса, на которые домножается скор документа от каждого ретривера
)

ensemble_retriever.get_relevant_documents(
    "Кого убили раздетой на крыльце?"
)

In [ ]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """
Answer the question based only on the following context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
chain.invoke("Кого убили раздетой на крыльце?")

In [26]:
from tqdm import tqdm

answers = [] # Список, где будем хранить ответы модели

for text_input in tqdm(doc1['question']):
    answer = chain.invoke(text_input)
    answers.append(answer) # Добавляем ответ в список
     # Для отладки. Уберите, когда убедитесь, что на одном примере работает
print(answers)    

100%|██████████| 15/15 [00:14<00:00,  1.00it/s]

['Раздетую на крыльце убили Василису Егоровну.', 'Маша остановилась в Царском Селе в Петербурге.', 'Гринев и Савельич сбились с дороги из-за бурана, который застал их в пути.', 'Над Машей издевался Швабрин и морил ее голодом.', 'Гринева приговорили к ссылке в Сибирь.', 'В семье Петра Гринева было девять детей, но все, кроме Петра, умерли в младенчестве.', 'Петра Гринева воспитывал дядька Савельич.', 'Гринев попросил полсотни казаков, чтобы очистить Белогорскую крепость.', 'Белогорская крепость находилась в 40 верст от города.', 'Дочку коменданта звали Маша.', 'В предоставленном контексте нет информации о том, что Петя подарил вожатому.', 'Отец велит Петру беречь ему «платье снову, а честь смолоду».', 'Пугачев оставил комендантом Белогорской крепости Швабрина.', 'Швабрина перевели в крепость за дуэль.', 'Савельич просил пощадить Гринева у Пугачева.']


In [27]:
doc1['answer'] = answers
doc1

,question,answer
0,Кого убили раздетой на крыльце?,Раздетую на крыльце убили Василису Егоровну.
1,Где Маша остановилась в Петербурге?,Маша остановилась в Царском Селе в Петербурге.
2,Почему Гринев и Савельич сбились с дороги?,Гринев и Савельич сбились с дороги из-за буран...
3,Кто издевался над Машей и морил ее голодом?,Над Машей издевался Швабрин и морил ее голодом.
4,Куда приговорили отправить Гринева в ссылку?,Гринева приговорили к ссылке в Сибирь.
5,Сколько детей было в семье Петра Гринева?,"В семье Петра Гринева было девять детей, но вс..."
6,Кто воспитывал Петра Гринева?,Петра Гринева воспитывал дядька Савельич.
7,"Сколько казаков попросил Гринев, чтобы очистит...","Гринев попросил полсотни казаков, чтобы очисти..."
8,Как далеко находилась от города Белогорская кр...,Белогорская крепость находилась в 40 верст от ...
9,Как звали дочку коменданта?,Дочку коменданта звали Маша.


In [28]:
doc1.to_csv('4_1_11_solution.csv', index=False)